<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Aplicada/blob/main/AgenteDataBaseChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Chatbot para Datos Estructurados con Ollama
Este script implementa un chatbot conversacional para análisis de datos estructurados (como CSVs) que:
1. Utiliza Ollama como motor LLM local
2. Proporciona una interfaz web simple con Gradio
3. Permite hacer preguntas sobre los datos mediante lenguaje natural
4. Funciona directamente en Google Colab
5. El LLM genera y ejecuta código Python para responder las consultas.

Por [Camilo Vega](https://www.linkedin.com/in/camilo-vega-169084b1/), Consultor en IA

In [ ]:
# --- 0. Configuración Inicial ---
# Nombre del modelo Ollama a utilizar (cambia esto para usar un modelo diferente)
# Asegúrate de que este modelo exista en Ollama y sea adecuado para tareas de generación de código.
# Modelos como 'codellama', 'deepseek-coder', 'llama3' suelen ser buenos para esto.
# 'llama3' es una buena opción general.
ollama_model_name = "llama3" # Prueba con llama3, o "codellama" si quieres enfocarte en código
# ollama_model_name = "llama2:7b" # Puedes volver a llama2 si prefieres

# Nombre del archivo CSV si decides usar uno real (deja None para usar datos simulados)
# Para usar un archivo real:
# 1. Sube tu archivo CSV a tu sesión de Colab.
# 2. Cambia esta variable a "nombre_de_tu_archivo.csv"
csv_file_name = None # Cambia a "tu_archivo.csv" para usar un archivo real

# --- 1. Instalación de Librerías y Ollama ---

print("--- Instalando librerías Python ---")
# Instalamos/actualizamos pandas, numpy, ollama y gradio
# Usamos --upgrade para asegurar versiones compatibles y -q para reducir el output
# ¡pandas y numpy ya deberían estar bien de la vez anterior, pero --upgrade no hace daño!
!pip install --upgrade pandas numpy ollama gradio openpyxl -q

# **Verificación de importaciones**
print("\n--- Verificando importaciones básicas ---")
try:
    import pandas as pd
    import numpy as np
    import ollama
    import gradio as gr
    import matplotlib.pyplot as plt # Para plots generados por código
    import io
    import sys # Importar sys para capturar stdout
    print("¡pandas, numpy, ollama, gradio, matplotlib, io y sys importados correctamente!")
except ImportError as e:
    print(f"ERROR CRÍTICO: No se pudo importar una librería necesaria: {e}")
    print("Asegúrate de que las instalaciones se completaron sin errores graves.")
    # import sys; sys.exit("Fallo en importaciones básicas.") # Descomentar si quieres detenerte aquí
except Exception as e:
    print(f"ERROR inesperado durante la importación: {e}")
    # import sys; sys.exit("Fallo en importaciones básicas.") # Descomentar si quieres detenerte aquí


# Instalar Ollama en Colab (solo es necesario ejecutar una vez por sesión)
print("\n--- Instalando Ollama ---")
!curl -fsSL https://ollama.com/install.sh | sh

# --- 2. Iniciar el servidor de Ollama en segundo plano ---
print("\n--- Iniciando servidor Ollama ---")
!pkill ollama || true # Asegura que no hay procesos ollama corriendo
!nohup /usr/local/bin/ollama serve > ollama_output.log 2>&1 & # Ejecuta en segundo plano

# --- 3. Esperar a que el servidor de Ollama inicie completamente ---
import time
print("Esperando a que el servidor Ollama esté listo (15 segundos)...")
time.sleep(15) # Ajusta este tiempo si ves errores de conexión iniciales

# --- 4. Verificar que el servidor esté respondiendo ---
print("\n--- Verificando servidor Ollama ---")
!curl -s http://localhost:11434/api/tags || echo "El servidor Ollama no está respondiendo. Revisa los logs o aumenta el tiempo de espera."

# --- 5. Descargar el modelo especificado ---
print(f"\n--- Descargando modelo {ollama_model_name} desde Ollama ---")
# Si cambiaste ollama_model_name, esto descargará el nuevo. Si ya lo tienes, será rápido.
!ollama pull {ollama_model_name}
print(f"Modelo {ollama_model_name} descargado (o ya disponible).")

# --- 6. Preparar los datos (Simulados o cargados desde CSV) ---
# pandas y os ya deberían estar importados desde la verificación inicial
import os

print("\n--- Cargando datos ---")

df = None # Inicializar el dataframe a None

if csv_file_name and os.path.exists(csv_file_name):
    # Si se especificó un archivo CSV y existe, cargarlo
    try:
        df = pd.read_csv(csv_file_name)
        print(f"Datos cargados desde '{csv_file_name}'. Filas: {len(df)}, Columnas: {len(df.columns)}")
    except Exception as e:
        print(f"Error al cargar el archivo CSV '{csv_file_name}': {e}")
        print("Se procederá a usar datos simulados.")
        csv_file_name = None # Revertir para usar datos simulados si falla la carga
else:
    if csv_file_name:
        print(f"Archivo '{csv_file_name}' no encontrado.")
        print("Se procederá a usar datos simulados.")
        csv_file_name = None # Revertir para usar datos simulados si no se encuentra

# Si no se cargó un archivo CSV, crear datos simulados
if df is None:
    print("Creando datos económicos simulados...")
    data = {
        'Pais': ['Argentina', 'Brasil', 'Chile', 'Colombia', 'Mexico', 'España', 'Francia', 'Alemania', 'Italia', 'Canada', 'USA', 'China', 'India', 'Japon', 'Australia'],
        'Continente': ['Sudamérica', 'Sudamérica', 'Sudamérica', 'Sudamérica', 'Norteamérica', 'Europa', 'Europa', 'Europa', 'Europa', 'Norteamérica', 'Norteamérica', 'Asia', 'Asia', 'Asia', 'Oceanía'],
        'PIB_miles_millones_USD': [632, 2080, 317, 350, 1320, 1460, 2700, 4260, 2060, 1980, 25500, 17700, 3500, 4900, 1700],
        'Poblacion_millones': [45.8, 214.3, 19.5, 51.5, 128.9, 47.4, 67.7, 83.2, 59.0, 38.2, 331.9, 1450.0, 1400.0, 125.8, 25.7],
        'Anio': [2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023]
    }
    df = pd.DataFrame(data)
    print("Datos simulados creados.")

print("\nPrimeras 5 filas del DataFrame:")
print(df.head().to_markdown(index=False)) # Mostrar como markdown para mejor legibilidad en Colab
print(f"\nColumnas y tipos de datos:")
df.info() # Usar df.info() directamente para mostrar en la consola de Colab


# --- 7. Funciones para interactuar con Ollama y ejecutar código ---

# Pre-generar la descripción del DataFrame que se enviará al LLM
# Esto se hace una vez después de cargar o crear el DataFrame
# Usamos io.StringIO() para capturar el output de df.info() como string para el LLM
buffer = io.StringIO()
df.info(verbose=True, buf=buffer)
dataframe_description = f"""Tienes acceso a un pandas DataFrame llamado 'df'.
Su información (columnas, tipos, no-nulos) es:
{buffer.getvalue()}

Primeras filas:
{df.head().to_string()}
"""

def generate_python_code(prompt, dataframe_info=dataframe_description, model=ollama_model_name):
    """
    Envía un prompt a Ollama pidiendo generar código Python para analizar el DataFrame.
    Devuelve solo el código Python generado.
    """
    # Este es el prompt clave para dirigir al LLM
    system_prompt = f"""Eres un asistente experto en análisis de datos usando pandas, numpy y matplotlib.
    Tu tarea es generar **SOLO CÓDIGO PYTHON** para responder preguntas sobre un pandas DataFrame llamado 'df'.
    El DataFrame tiene la siguiente estructura y tipos de datos:
    {dataframe_info}

    Reglas estrictas para tu respuesta:
    1.  **SOLO RESPONDE CON CÓDIGO PYTHON.** No incluyas texto explicativo, markdown extra (```), ni prompts (>>>).
    2.  El código debe ser una expresión o un conjunto de sentencias que se puedan ejecutar directamente.
    3.  Las librerías `pandas` (como `pd`), `numpy` (como `np`) y `matplotlib.pyplot` (como `plt`) ya están importadas y disponibles en el entorno. No las importes de nuevo.
    4.  Si la respuesta es un valor, DataFrame pequeño o texto, usa `print()` para mostrar el resultado.
    5.  Si se pide una visualización, genera el código completo de matplotlib/seaborn para crear la figura. **La última línea de tu código debe ser la figura** (ej: `fig`) para que Gradio la muestre. No incluyas `plt.show()`.
    6.  Asegúrate de que el código es válido y se ejecuta sin errores con el DataFrame proporcionado.
    7.  Siempre que sea posible, la última línea de código debe ser el resultado (valor impreso, DataFrame impreso, o objeto figura).
    """

    user_prompt = f"Pregunta del usuario: {prompt}"

    print(f"\n--- Enviando prompt a {model} para generar código ---")
    # Usamos la librería ollama para interactuar con el servidor local
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_prompt},
            ],
            options={
                "temperature": 0.0, # Mantener bajo para código preciso
                # "num_predict": 512 # Opcional: limitar longitud si es necesario
            }
        )
        # Asegurarse de que solo se extrae el contenido de la respuesta del modelo
        code = response['message']['content'].strip()

        # A veces, los LLMs pueden incluir markdown a pesar de las instrucciones estrictas.
        # Intentar limpiar bloques de código markdown si aparecen.
        if code.startswith("```python"):
            code = code.replace("```python", "").strip()
            if code.endswith("```"):
                code = code[:-3].strip()
        elif code.startswith("```"): # Otros bloques de código
             code = code.replace("```", "").strip()


        print("Código Python generado:")
        print("--- INICIO CÓDIGO ---")
        print(code)
        print("--- FIN CÓDIGO ---")
        return code
    except Exception as e:
        print(f"Error al llamar a Ollama para generar código: {e}")
        return None

# Asegurarnos de que numpy y matplotlib están en el entorno de ejecución para exec
import numpy as np # Usar np consistentemente
import matplotlib.pyplot as plt

def execute_python_code(code, dataframe):
    """
    Ejecuta el código Python generado contra el DataFrame.
    Devuelve el resultado de la ejecución (texto o plot).
    """
    print("\n--- Ejecutando código Python ---")
    # Usamos exec para ejecutar código en un entorno controlado
    # Creamos un diccionario para el entorno de ejecución, incluyendo el DataFrame 'df' y librerías comunes
    execution_env = {'df': dataframe, 'pd': pd, 'np': np, 'plt': plt} # Usamos 'np' y 'plt'

    # Redirigir stdout para capturar output de print
    old_stdout = sys.stdout
    redirected_output = io.StringIO()
    sys.stdout = redirected_output

    # Usamos un diccionario para capturar el resultado si el código lo asigna a una variable 'result'
    # o si la última expresión no es print.
    result_holder = {'_result': None}
    code_to_exec = f"import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n{code}\n\n_result = None\ntry:\n    _result = eval(compile(code_to_exec, '<string>', 'eval'), execution_env)\nexcept:\n    pass # Could not evaluate as single expression"
     # ^^ Intentamos envolverlo para capturar la última expresión, pero esto puede ser frágil.
     # Un enfoque más robusto es confiar en que el LLM use print() o retorne una figura.

    # Simplificamos la ejecución para confiar más en el `print` del código generado o en el retorno de la figura
    try:
        # Ejecutar el código en el entorno preparado
        exec(code, execution_env)

        # Restaurar stdout
        sys.stdout = old_stdout
        output_text = redirected_output.getvalue().strip() # Obtener output impreso

        # Intentar detectar si se generó un plot
        fig = None
        try:
            # Si hay figuras activas y no se cerraron explícitamente
            # get_fignums() devuelve una lista de ids de figuras activas
            if plt.get_fignums():
                 # Asumimos que la última figura creada es la relevante
                 fig = plt.gcf() # Get Current Figure
                 print("(Código ejecutado, se detectó una figura de matplotlib)") # Mensaje para el log
                 # Es buena práctica limpiar las figuras después de obtener la que queremos
                 # plt.close('all') # Podríamos cerrarlas aquí, o después de retornarlas en el handler de Gradio
        except Exception as fig_e:
             print(f"Advertencia al verificar figuras de matplotlib: {fig_e}")
             pass # No fallar si falla la detección de figura

        # Determinar el resultado a devolver
        if fig:
            # Si hay una figura, devolver la figura (Gradio la mostrará en el componente Plot)
            return fig
        elif output_text:
            # Si hay output de texto, devolverlo (Gradio lo mostrará en el componente Textbox)
            return output_text
        else:
            # Si no hay ni figura ni output de texto
            # Podríamos intentar devolver el valor de result_holder['_result'] si lo implementamos,
            # pero basándonos en las instrucciones del prompt, el LLM debería imprimir o generar figura.
            # Retornar un mensaje informativo.
            return "Código ejecutado. No se produjo output de texto ni figura visible. Revisa el prompt o el código generado."

    except Exception as e:
        # Restaurar stdout si ocurrió un error antes de restaurarlo
        sys.stdout = old_stdout
        # Limpiar figuras si hubo un error para evitar que interfieran en la siguiente ejecución
        plt.close('all')
        print(f"Error al ejecutar el código: {e}")
        # Devolver el mensaje de error a la interfaz
        # Incluimos el código ejecutado para depuración en la interfaz
        return f"ERROR al ejecutar el código:\n{e}\n\nCódigo ejecutado:\n```python\n{code}\n```"


# --- 8. Integración con Gradio ---

# Función que Gradio llamará
def chatbot_response(user_input):
    """Procesa la entrada del usuario, genera y ejecuta código, y devuelve el resultado."""
    global df # Asegúrate de que df está accesible (definido en la Celda 6)

    if df is None:
        return "Error: DataFrame no cargado. Verifica la configuración.", None # Retornar tupla (texto, None)

    # Opciones especiales para la interfaz (opcional, puedes quitar si no las necesitas)
    if user_input.lower() == 'mostrar dataframe info':
         buffer = io.StringIO()
         df.info(verbose=True, buf=buffer)
         return buffer.getvalue(), None # Devuelve texto y None para el plot
    if user_input.lower() == 'mostrar primeras filas':
         return df.head().to_markdown(index=False), None # Devuelve texto y None para el plot
    if user_input.lower() == 'mostrar datos simulados':
         return df.to_markdown(index=False), None # Devuelve texto y None para el plot


    # Generar el código Python usando Ollama
    code_to_execute = generate_python_code(user_input, dataframe_description, model=ollama_model_name)

    # Limpiar figuras existentes antes de ejecutar un nuevo código que podría crear plots
    plt.close('all')

    if code_to_execute:
        # Ejecutar el código generado
        execution_result = execute_python_code(code_to_execute, df)

        # Gradio puede manejar la distinción entre texto y figura si la función retorna el tipo correcto
        # Si execution_result es un string, Gradio lo pondrá en el primer output (Textbox)
        # Si execution_result es un Figure, Gradio lo pondrá en el segundo output (Plot)
        # Debemos devolver una tupla (texto, plot) donde uno es el resultado y el otro es None
        if isinstance(execution_result, plt.Figure):
            # Limpiar la figura *después* de que Gradio la haya procesado
            # plt.close(execution_result) # Esto puede causar problemas si Gradio aún la necesita
            # Gradio gestiona la visualización, no siempre es necesario cerrarla manualmente inmediatamente aquí.
            # La detección en execute_python_code y el plt.close('all') al inicio de esta función ayudan.
            return None, execution_result # No hay texto, retorna la figura
        elif isinstance(execution_result, str):
            return execution_result, None # Retorna el texto, no hay figura
        else:
            # Manejar otros tipos inesperados
            return f"Resultado inesperado del código: {type(execution_result)}. Resultado: {execution_result}", None

    else:
        # Si no se pudo generar código
        return "No se pudo generar código para esta pregunta. Intenta de nuevo.", None


print("\n--- Configurando interfaz Gradio ---")

# Crear la interfaz Gradio
if df is not None: # Solo si el DataFrame se cargó correctamente
    interface = gr.Interface(
        fn=chatbot_response, # La función a llamar
        inputs=gr.Textbox(label="Tu pregunta sobre los datos o comando especial"), # Un campo de texto para la entrada
        outputs=[
            gr.Textbox(label="Respuesta de texto", interactive=False, visible=True), # Campo para respuestas de texto, siempre visible
            gr.Plot(label="Gráfico", visible=True) # Campo para gráficos, siempre visible
            # Gradio ocultará automáticamente los componentes si el valor retornado es None
        ],
        title="CSV Chatbot con Ollama y Gradio",
        description=f"""Haz preguntas sobre los datos del DataFrame.
        Columnas: {', '.join(df.columns)}. Modelo LLM: {ollama_model_name}<br>
        Puedes usar comandos especiales como: `mostrar dataframe info`, `mostrar primeras filas`, `mostrar datos simulados`.
        """,
        live=False, # Set to True for real-time updates as you type (puede aumentar carga en Colab)
        allow_flagging="never" # Evitar botón de "Flag"
    )

    print("\n--- Iniciando interfaz Gradio ---")
    # share=True genera un enlace público de Hugging Face temporal.
    # debug=True muestra logs de Gradio (útil para depurar).
    # El puerto 7860 es el predeterminado de Gradio, asegúrate de que no esté en uso.
    # Usamos `inline=False` para forzar que el enlace se muestre claramente, no incrustado.
    interface.launch(share=True, debug=True, inline=False)

    print("\n--- Interfaz Gradio iniciada. Busca el enlace 'Public URL' en el output de la celda. ---")
else:
    print("\nDataFrame no cargado. No se puede iniciar la interfaz Gradio.")



--- Instalando librerías Python ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 750.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflict

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(



--- Iniciando interfaz Gradio ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f10383ddf2bc52308c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)

--- Enviando prompt a llama3 para generar código ---
Código Python generado:
--- INICIO CÓDIGO ---
print(df)
--- FIN CÓDIGO ---

--- Ejecutando código Python ---

--- Enviando prompt a llama3 para generar código ---
Código Python generado:
--- INICIO CÓDIGO ---
print(df['PIB_miles_millones_USD'].mean())
--- FIN CÓDIGO ---

--- Ejecutando código Python ---

--- Enviando prompt a llama3 para generar código ---
Código Python generado:
--- INICIO CÓDIGO ---
print(df.groupby('Continente')['PIB_miles_millones_USD'].sum().plot(kind='bar', figsize=(10,6), t